# Web Server

In [1]:
import flask

import werkzeug

import psycopg2

import json


# Lab: Web Server With Static Content

## Flask is generally informally called "a web server written in Python"; Flask is really easy for those who know Python to develop a dynamic web site and/or web api server;  

## Flask is offically called a "micro web framework" or "microframework" or more generically "application server", as it requires an actual web server and WSGI (Web Server Gateway Interface); 

## As Flask requires a web server and WSGI to function, Flask comes with Werkzeug, which is a very lightweight, combination web server and WSGI in one package;  Werkzeug warns that it is a lightweight product, intended for development purposes only - not secure for public hosting

## For Production, the most common configurations to replace Werkzeug are (we will do this next week): 

* for the web server:  nginx (pronounced "engine X"); the web server will also serve all static content instead of Flask
* for the WSGI: either Green Unicorn (aka "gunicorn" pronounced "gee-unicorn" or "gun-uh-corn") which is the most popular, or uWSGI which is a bit older and not as popular these days;  

## Simplist Flask application to serve static content;  by default, Flask will serve static content from the static directory; contrary to the behavior of web servers such as nginx or Apache, it does not automatically route a directory to index.html

In [2]:
app = flask.Flask(__name__)


## For now, we will just use Werkzeug for the web server and the WSGI;  hostname of "0.0.0.0" allows it to bind to external address, so you can surf it from your laptop; ssl_context is how we specify a certificate and a key for https to allow our content to be encryped across the public internet;  we are using a self-signed certificate, so the browser will generate a warning, but will still encrypt; https uses port 443

## This is an infinite loop, use menu bar => Kernel => Interrupt to stop

In [4]:
werkzeug.serving.run_simple(hostname="0.0.0.0", 
                            port=443, 
                            application=app,
                            ssl_context=("w205_cert.pem","w205.key"),
                            use_debugger=True)

 * Running on https://0.0.0.0:443/ (Press CTRL+C to quit)
75.24.123.6 - - [16/Nov/2021 08:00:38] "GET /static/index.html HTTP/1.1" 200 -
75.24.123.6 - - [16/Nov/2021 08:02:42] "GET /static/bootstrap-5.1.3-examples/cheatsheet/index.html HTTP/1.1" 200 -
75.24.123.6 - - [16/Nov/2021 08:02:42] "GET /static/bootstrap-5.1.3-examples/cheatsheet/cheatsheet.css HTTP/1.1" 200 -
75.24.123.6 - - [16/Nov/2021 08:02:42] "GET /static/bootstrap-5.1.3-examples/assets/dist/css/bootstrap.min.css HTTP/1.1" 200 -
75.24.123.6 - - [16/Nov/2021 08:02:42] "GET /static/bootstrap-5.1.3-examples/assets/dist/js/bootstrap.bundle.min.js HTTP/1.1" 200 -
75.24.123.6 - - [16/Nov/2021 08:02:42] "GET /static/bootstrap-5.1.3-examples/cheatsheet/cheatsheet.js HTTP/1.1" 200 -
75.24.123.6 - - [16/Nov/2021 08:02:43] "GET /static/bootstrap-5.1.3-examples/assets/brand/bootstrap-logo-white.svg HTTP/1.1" 200 -
75.24.123.6 - - [16/Nov/2021 08:02:55] "GET /static/bootstrap-5.1.3-examples/blog/index.html HTTP/1.1" 200 -
75.24.123.6 

## Assuming you setup the firewall rules earlier this semester to allow inbound traffic on port 443, you should be able to surf this website from your laptop or desktop;

## In your browser (replace the xxxxxx with the external IP address of your VM in AWS):

## https://xxxxxxx/static/index.html


## This is not very convenience to have to put /static/index.html into our URL, and is contrary to the behavior of all other web browsers; change our Flask application to mimic the standard behavior of static files at / and automatically serving index.html when the file name is not given; this change will also make it a lot easier next week when we switch to nginx and Green Unicorn

In [5]:
app = flask.Flask(__name__,
                  static_url_path="")

@app.route("/")
def landing_page():
    return flask.send_from_directory("static","index.html") 


In [6]:
werkzeug.serving.run_simple(hostname="0.0.0.0", 
                            port=443, 
                            application=app,
                            ssl_context=("w205_cert.pem","w205.key"),
                            use_debugger=True)

 * Running on https://0.0.0.0:443/ (Press CTRL+C to quit)
75.24.123.6 - - [16/Nov/2021 08:08:03] "GET / HTTP/1.1" 200 -


## Now we can surf it simply using (replace the xxxxx with the IP address of your VM in AWS):

## https://xxxxxx

## Repeat using your phone and a tablet (if you have one)

## You try it - Look through the Bootstrap examples and get a feel for the different elements and design available in modern RWD (responsive web design);  Be sure and also try each one on your phone and tablet (if you have one) and see how RWD plays out on the different devices and sizes


# Lab: Adding Dynamic Content to Our Web Server

In [7]:
connection = psycopg2.connect(
    user = "postgres",
    password = "ucb",
    host = "postgres",
    port = "5432",
    database = "postgres"
)

In [8]:
cursor = connection.cursor()

In [9]:
def my_query_products():
    "query the products from Postgres and return a Python list of products"
    
    connection.rollback()

    query = """
    
    select p.product_id, p.description, sum(quantity), sum(quantity * 12)
    from products p
         join line_items l
             on p.product_id = l.product_id
    group by p.product_id, p.description
    order by p.product_id
    
    """
    
    cursor.execute(query)
    
    rows = cursor.fetchall()

    connection.rollback()
    
    products_list = []
    
    for row in rows:
        
        products_list.append([row[0], row[1], f'{row[2]:,}', f'{row[3]:,}'])
        
    return(products_list)

In [10]:
app = flask.Flask(__name__,
                  static_url_path="")

@app.route("/")
def landing_page():
    return flask.send_from_directory("static", "index.html")

@app.route("/products")
def products():
    
    products_list = my_query_products()
    
    return(flask.render_template("products.html", products_list=products_list))

@app.route("/products/bootstrap")
def products_bootstrap():
    
    products_list = my_query_products()
    
    return(flask.render_template("products_bootstrap.html", products_list=products_list))


In [11]:
werkzeug.serving.run_simple(hostname="0.0.0.0", 
                            port=443, 
                            application=app,
                            ssl_context=("w205_cert.pem","w205.key"),
                            use_debugger=True)

 * Running on https://0.0.0.0:443/ (Press CTRL+C to quit)
75.24.123.6 - - [16/Nov/2021 08:24:25] "GET /static/index.html HTTP/1.1" 404 -
75.24.123.6 - - [16/Nov/2021 08:24:38] "GET / HTTP/1.1" 200 -
75.24.123.6 - - [16/Nov/2021 08:24:51] "GET /static/index.html HTTP/1.1" 404 -
75.24.123.6 - - [16/Nov/2021 08:24:57] "GET / HTTP/1.1" 200 -
75.24.123.6 - - [16/Nov/2021 08:25:03] "GET / HTTP/1.1" 200 -
75.24.123.6 - - [16/Nov/2021 08:26:03] "GET / HTTP/1.1" 200 -
75.24.123.6 - - [16/Nov/2021 08:26:10] "GET /products HTTP/1.1" 200 -
75.24.123.6 - - [16/Nov/2021 08:26:31] "GET /products/bootstrap HTTP/1.1" 200 -
75.24.123.6 - - [16/Nov/2021 08:26:44] "GET / HTTP/1.1" 200 -
75.24.123.6 - - [16/Nov/2021 08:26:47] "GET / HTTP/1.1" 200 -
75.24.123.6 - - [16/Nov/2021 08:26:48] "GET / HTTP/1.1" 200 -
75.24.123.6 - - [16/Nov/2021 08:26:51] "GET /bootstrap-5.1.3-examples/blog/index.html HTTP/1.1" 200 -
75.24.123.6 - - [16/Nov/2021 08:26:52] "GET /bootstrap-5.1.3-examples/assets/dist/css/bootstrap.mi

## The route /products will query the products and return HTML without any formatting

## http://xxxxx/products

## The route /products/bootstap will query the products and return HTML with Bootstrap formatting:

## http://xxxxx/products/bootstrap

## Try both on your laptop, desktop, phone, tablet, etc.

## You try it - add a route /stores that returns a web page with an HTML table with store id, city, total sales; solutions are in web_server_solutions and templates/stores_solutions.html

## You try it - add a route /stores/bootstrap that formats the results using Bootstrap; solutions are in web_server_solutions and templates/stores_bootstrap_solutions.html

## Try both on your laptop, desktop, phone, tablet, etc.


# Lab: Flask - Client-Side GET Calls (Without Parameters) Using Python

In [12]:
app = flask.Flask(__name__,
                  static_url_path="")

@app.route("/")
def landing_page():
    return flask.send_from_directory("static", "index.html")

@app.route("/products")
def products():
    
    products_list = my_query_products()
    
    return(flask.render_template("products.html", products_list=products_list))

@app.route("/products/bootstrap")
def products_bootstrap():
    
    products_list = my_query_products()
    
    return(flask.render_template("products_bootstrap.html", products_list=products_list))

@app.route("/api/products", methods=["GET"])
def api_products():
    
    products_list = my_query_products()
    
    products_json_list = []
    
    for product in products_list:
        
        p = {}
        p["product_id"] = str(product[0])
        p["product_name"] = product[1]
        p["quantity"] = str(product[2])
        p["total_sales"] = str(product[3])
        
        products_json_list.append(p)
        
    return(json.dumps(products_json_list))


In [13]:
werkzeug.serving.run_simple(hostname="0.0.0.0", 
                            port=443, 
                            application=app,
                            ssl_context=("w205_cert.pem","w205.key"),
                            use_debugger=True)

 * Running on https://0.0.0.0:443/ (Press CTRL+C to quit)
75.24.123.6 - - [16/Nov/2021 08:38:48] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [16/Nov/2021 08:41:21] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [16/Nov/2021 08:43:31] "GET /api/products HTTP/1.1" 200 -


## The client side is in the jupyter notebook web_client


# Lab: Flask - Server-Side GET Calls (Without Parameters) Using Python

## The server side code is in the previous section


## You try it - add an API call for route /api/stores which returns the stores query results as JSON; solution is in web_server_solutions and web_client_solutions


# Lab: Flask - Client-Side GET Calls (With Parameters) Using Python 

In [14]:
app = flask.Flask(__name__,
                  static_url_path="")

@app.route("/")
def landing_page():
    return flask.send_from_directory("static", "index.html")

@app.route("/products")
def products():
    
    products_list = my_query_products()
    
    return(flask.render_template("products.html", products_list=products_list))

@app.route("/products/bootstrap")
def products_bootstrap():
    
    products_list = my_query_products()
    
    return(flask.render_template("products_bootstrap.html", products_list=products_list))

@app.route("/api/products", methods=["GET"])
def api_products():
    
    products_list = my_query_products()
    
    products_json_list = []
    
    product_parameter = flask.request.args.get("product")
    
    for product in products_list:
        
        if product_parameter == None or (product_parameter != None and product_parameter == str(product[0])):
            
            p = {}
            p["product_id"] = str(product[0])
            p["product_name"] = product[1]
            p["quantity"] = str(product[2])
            p["total_sales"] = str(product[3])

            products_json_list.append(p)
        
    return(json.dumps(products_json_list))
    


In [15]:
werkzeug.serving.run_simple(hostname="0.0.0.0", 
                            port=443, 
                            application=app,
                            ssl_context=("w205_cert.pem","w205.key"),
                            use_debugger=True)

 * Running on https://0.0.0.0:443/ (Press CTRL+C to quit)
127.0.0.1 - - [16/Nov/2021 09:24:55] "GET /api/products?product=3 HTTP/1.1" 200 -
127.0.0.1 - - [16/Nov/2021 09:37:45] "GET /api/products HTTP/1.1" 200 -


## The client side is in the jupyter notebook web_client


# Lab: Flask - Server-Side GET Calls (With Parameters) Using Python

## The server side code is in the previous section


## You try it - modify an API call for route /api/stores to have the option of passing a store parameter; solution is in web_server_solutions and web_client_solutions


# Lab: Flask - Client-Side POST Calls Using Python

In [16]:
app = flask.Flask(__name__,
                  static_url_path="")

@app.route("/")
def landing_page():
    return flask.send_from_directory("static", "index.html")

@app.route("/products")
def products():
    
    products_list = my_query_products()
    
    return(flask.render_template("products.html", products_list=products_list))

@app.route("/products/bootstrap")
def products_bootstrap():
    
    products_list = my_query_products()
    
    return(flask.render_template("products_bootstrap.html", products_list=products_list))

@app.route("/api/products", methods=["GET","POST"])
def api_products():
    
    products_list = my_query_products()
    
    products_json_list = []
    
    if flask.request.method == "GET":
        
        product_parameter = flask.request.args.get("product")
        
    elif flask.request.method == "POST":
        
        product_parameter = flask.request.form['product']
    
    for product in products_list:
        
        if product_parameter == None or (product_parameter != None and product_parameter == str(product[0])):
            
            p = {}
            p["product_id"] = str(product[0])
            p["product_name"] = product[1]
            p["quantity"] = str(product[2])
            p["total_sales"] = str(product[3])

            products_json_list.append(p)
        
    return(json.dumps(products_json_list))
    


In [17]:
werkzeug.serving.run_simple(hostname="0.0.0.0", 
                            port=443, 
                            application=app,
                            ssl_context=("w205_cert.pem","w205.key"),
                            use_debugger=True)

 * Running on https://0.0.0.0:443/ (Press CTRL+C to quit)
127.0.0.1 - - [16/Nov/2021 09:48:06] "POST /api/products HTTP/1.1" 200 -


## The client side is in the jupyter notebook web_client


# Lab: Flask - Server-Side POST Calls Using Python

## The server side code is in the previous section


## You try it - modify an API call for route /api/stores to have the option of passing a store parameter using JSON in a POST method; solution is in web_server_solutions and web_client_solutions
